<a href="https://colab.research.google.com/github/NikithaAshok/LegalTextSummarization/blob/main/abstractive-methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!git clone https://github.com/NikithaAshok/LegalTextSummarization.git

Cloning into 'LegalTextSummarization'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 4.00 KiB | 2.00 MiB/s, done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#T5 is an encoder-decoder model. Converts all language problems into text-to-text format.

# from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
import transformers
# transformers.cache.clear_cache()
import PyPDF2
from PyPDF2 import PdfReader
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import string

#creating a pdf file object
pdf = open("/content/drive/MyDrive/example.pdf","rb")

#creating a pdf reader object
pdf_reader = PyPDF2.PdfReader(pdf)

#checking number of pages in a pdf file
#print("Number of pages in the pdf ",len(pdf_reader.pages))

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


#stemming tokens 

#function for tokenization and removing stopwords
def remove_stop_words(text):
    punct_removed_text = text.translate(str.maketrans('','',string.punctuation))
    words = nltk.word_tokenize(punct_removed_text)
    words = [word for word in words if word.lower() not in stop_words]
    return " ".join(words)


#function for stemming
def stemming(text):
    tokens = text.split(' ')

    #defining a Stemmer
    stemmer = PorterStemmer()

    #stem the tokens 
    stemmed_tokens = []


    for token in tokens:
        stemmed_token = stemmer.stem(token)
        stemmed_tokens.append(stemmed_token)

    #join the stemmed tokens back into a string
    stemmed_text = ' '.join(stemmed_tokens)

    return stemmed_text

#T5 Model 
# t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')
# t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

#BART Model
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

for i in range(0,len(pdf_reader.pages)):
    page = pdf_reader.pages[i]
    extracted_text = page.extract_text() # extraction happens here
    text_without_stopwords = remove_stop_words(extracted_text) #removing stopwords (function called)
    sentences = sent_tokenize(extracted_text)
    text = "summarize:" + extracted_text

#     #T5 MODEL -----------
#     #converting input sequence to input-ids through process of encoding - encode()
#     input_ids = t5_tokenizer.encode(text,return_tensors='pt',max_length=512)
#     #generate() method returns a sequence of ids corresponding to the summary
#     summary_ids = t5_model.generate(input_ids)
#     #using decode() function to generate summary text from the above ids
#     t5_summary = t5_tokenizer.decode(summary_ids[0])
#     print(t5_summary)

    #BART MODEL ------------
    bart_inputs = bart_tokenizer.encode(extracted_text,return_tensors='pt', max_length=1024, truncation=True)
    bart_summary_ids = bart_model.generate(bart_inputs, num_beams=4, max_length=100, early_stopping=True)
#     bart_summary = bart_tokenizer.decode(bart_summary_ids[0],skip_special_tokens=True)
    bart_summary = bart_tokenizer.decode(bart_summary_ids[0],skip_special_tokens=True)
    print('summary',bart_summary)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


summary Self Attention Based Legal Document Summarization- A Technical Review. The study was conducted at the Atria Institute of Technology, Bangalore. The author is Nikitha A. A. Jain, a professor of computer science at the institute. The report was published in the online edition of the ATSR.
